<a href="https://colab.research.google.com/github/Fadhaa/Deep_STAT/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip -q install git+https://github.com/huggingface/transformers.git -U accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 34.2 MB/s eta 0:00:00


In [3]:
!pip install huggingface_hub

In [4]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00


In [5]:
from huggingface_hub import notebook_login

In [6]:
notebook_login()

In [7]:
import gc
import math
import re
import sys
import subprocess
import numpy as np
import pandas as pd
import random

In [8]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    StoppingCriteria,
    set_seed
)

In [9]:
import transformers
print(f"Transformers Version: {transformers.__version__}")
set_seed(14)

Transformers Version: 4.43.0.dev0


In [10]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [11]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-1.1-2b-it")

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [12]:
xi= np.random.normal(0, 1, 30)

In [13]:
yi=0.5+2*xi

In [14]:
x_str=','.join([str(z) for z in xi])

In [15]:
y_str=','.join([str(z) for z in yi])

In [16]:
prompt="for the data the dependent variable y is :"+y_str+ " and the independent variable x is :"+x_str+" find the maximum likelihood estimators of the parameters of the linear regression model of one variable. do not solve it. Only generate python code that give the final results ?"

In [17]:
def return_last_print(output, n):
    lines = output.strip().split('\n')
    if lines:
        return lines[n]
    else:
        return ""

In [18]:
def process_code(code, return_shell_output=False):

    def repl(match):
        if "real" not in match.group():
            return "{}{}".format(match.group()[:-1], ', real=True)')
        else:
            return "{}{}".format(match.group()[:-1], ')')
    code = re.sub(r"symbols\([^)]+\)", repl, code)

    if return_shell_output:
        code = code.replace('\n', '\n    ')
            # Add a try...except block
        code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)

    if not return_shell_output:
        print(code)
    with open('code.py', 'w') as fout:
        fout.write(code)

    batcmd = 'timeout 7 ' + sys.executable + ' code.py'
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
        return_value = return_last_print(shell_output, -1)
        print(shell_output)
        if return_shell_output:
            if return_value=='FAIL':
                CODE_STATUS = False
                return_value = return_last_print(shell_output, -2)
                if "not defined" in return_value:
                    return_value+='\nTry checking the formatting and imports'
            else:
                CODE_STATUS = True
            return return_value, CODE_STATUS
        code_output = round(float(eval(return_value))) % 1000
    except Exception as e:
        print(e,'shell_output')
        code_output = -1

    if return_shell_output:
        if code_output==-1:
            CODE_STATUS = False
        else:
            CODE_STATUS = True
        return code_output, CODE_STATUS


    return code_output

In [19]:
def process_text_output(output):
    result = output
    try:
        result_output = re.findall(r'\\boxed\{(\d+)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED FINAL', result_output)
        if not len(result_output):
            result_output = -1

        else:
            result_output = round(float(eval(result_output))) % 1000

    except Exception as e:
        print(e)
        print('ERROR PARSING TEXT')
        result_output = -1

    return result_output

In [20]:
from collections import defaultdict

In [21]:
import re
from collections import defaultdict
from collections import Counter

from numpy.random import choice
import numpy as np

tool_instruction = "\n\nTo accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!"
###Write the entire script covering all the steps. Make sure that all the variables are identified and the code is applicable.



#tool_instruction = " The answer should be given as a non-negative modulo 1000."
#tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

temperature = 0.9
top_p = 1.0

temperature_coding = 0.9
top_p_coding = 1.0

bbb=[]
total_results = {}
total_answers = {}
best_stats = {}
total_outputs = {}
question_type_counts = {}
starting_counts = (2,3)
stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"]


In [22]:
problem =prompt

In [23]:
problem

'for the data the dependent variable y is :3.602678222506126,0.658372049217643,0.8479530569123397,0.35532686766764654,-3.508658800344371,0.7893556263806322,-2.5023372445941905,0.9222189019020861,-0.6164101107521522,2.6690581297676106,0.12742175195198274,0.5293227495938443,-1.6511389388733662,1.7845041484337525,0.13932657734313764,1.7406005067503543,2.121948669236295,2.055871732795083,0.02502064411553584,-3.2838246906154196,-1.844243587783275,2.1724139426958806,2.5185468006643528,3.0861987218193314,-0.5986623311224653,-1.4634257299582547,2.3293455157704033,2.2461407565537512,0.5442222834102208,0.4917128329546021 and the independent variable x is :1.551339111253063,0.07918602460882154,0.17397652845616982,-0.07233656616617674,-2.0043294001721854,0.14467781319031608,-1.5011686222970952,0.21110945095104303,-0.5582050553760761,1.0845290648838053,-0.18628912402400863,0.014661374796922139,-1.075569469436683,0.6422520742168762,-0.18033671132843118,0.6203002533751771,0.8109743346181475,0.7779358

In [34]:
draw = """Below is a stat problem to generate python code to solve it:
\"{}\"
generate only python code by use python statsmodel that can be applied to given data\n\n.

Approach:"""


In [35]:
initail_message = draw.format(problem," \n")

In [36]:
initail_message

'Below is a stat problem to generate python code to solve it:\n"for the data the dependent variable y is :3.602678222506126,0.658372049217643,0.8479530569123397,0.35532686766764654,-3.508658800344371,0.7893556263806322,-2.5023372445941905,0.9222189019020861,-0.6164101107521522,2.6690581297676106,0.12742175195198274,0.5293227495938443,-1.6511389388733662,1.7845041484337525,0.13932657734313764,1.7406005067503543,2.121948669236295,2.055871732795083,0.02502064411553584,-3.2838246906154196,-1.844243587783275,2.1724139426958806,2.5185468006643528,3.0861987218193314,-0.5986623311224653,-1.4634257299582547,2.3293455157704033,2.2461407565537512,0.5442222834102208,0.4917128329546021 and the independent variable x is :1.551339111253063,0.07918602460882154,0.17397652845616982,-0.07233656616617674,-2.0043294001721854,0.14467781319031608,-1.5011686222970952,0.21110945095104303,-0.5582050553760761,1.0845290648838053,-0.18628912402400863,0.014661374796922139,-1.075569469436683,0.6422520742168762,-0.18

In [37]:
pro = f"User: {initail_message}"

In [38]:
model_inputs = tokenizer(pro, return_tensors='pt')

In [39]:
input_len = len(model_inputs['input_ids'][0])

In [40]:
USE_PAST_KEY=True
total_token=2040
temperature=0.9
top_p=1

In [31]:
import torch
from transformers import StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cpu") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            last_token = input_ids[0][-len(stop):]
            if torch.all(torch.eq(stop,last_token)):

                return True
        return False


stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"] #,
stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [41]:
generation_output = model.generate(**model_inputs,
                                   max_new_tokens=total_token,
                                   return_dict_in_generate=USE_PAST_KEY,
                                   do_sample = True,
                                   temperature = temperature,
                                   top_p = top_p,
                                   num_return_sequences=1)######, stopping_criteria = stopping_criteria)

In [42]:
if USE_PAST_KEY:
  output_ids = generation_output.sequences[0]
else:
  output_ids = generation_output[0]

In [43]:
decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)

In [44]:
decoded_output

'User: Below is a stat problem to generate python code to solve it:\n"for the data the dependent variable y is :3.602678222506126,0.658372049217643,0.8479530569123397,0.35532686766764654,-3.508658800344371,0.7893556263806322,-2.5023372445941905,0.9222189019020861,-0.6164101107521522,2.6690581297676106,0.12742175195198274,0.5293227495938443,-1.6511389388733662,1.7845041484337525,0.13932657734313764,1.7406005067503543,2.121948669236295,2.055871732795083,0.02502064411553584,-3.2838246906154196,-1.844243587783275,2.1724139426958806,2.5185468006643528,3.0861987218193314,-0.5986623311224653,-1.4634257299582547,2.3293455157704033,2.2461407565537512,0.5442222834102208,0.4917128329546021 and the independent variable x is :1.551339111253063,0.07918602460882154,0.17397652845616982,-0.07233656616617674,-2.0043294001721854,0.14467781319031608,-1.5011686222970952,0.21110945095104303,-0.5582050553760761,1.0845290648838053,-0.18628912402400863,0.014661374796922139,-1.075569469436683,0.6422520742168762

In [ ]:
pro

'User: Below is a math problem you are to solve (positive numerical answer):\n"Let 3x+5=14. Solve for x?."\nTo accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!\nWrite the entire script covering all the steps (use comments and document it well) and print the result. In the script function, determine what are the input values and what is the output result. Be aware that the final accepted result should satisfy all the problem conditions and constraints. \nAfter solving the problem, output the final numerical answer within \\boxed \n.\n\nApproach:'

In [45]:
print(f"{decoded_output}\n")

User: Below is a stat problem to generate python code to solve it:
"for the data the dependent variable y is :3.602678222506126,0.658372049217643,0.8479530569123397,0.35532686766764654,-3.508658800344371,0.7893556263806322,-2.5023372445941905,0.9222189019020861,-0.6164101107521522,2.6690581297676106,0.12742175195198274,0.5293227495938443,-1.6511389388733662,1.7845041484337525,0.13932657734313764,1.7406005067503543,2.121948669236295,2.055871732795083,0.02502064411553584,-3.2838246906154196,-1.844243587783275,2.1724139426958806,2.5185468006643528,3.0861987218193314,-0.5986623311224653,-1.4634257299582547,2.3293455157704033,2.2461407565537512,0.5442222834102208,0.4917128329546021 and the independent variable x is :1.551339111253063,0.07918602460882154,0.17397652845616982,-0.07233656616617674,-2.0043294001721854,0.14467781319031608,-1.5011686222970952,0.21110945095104303,-0.5582050553760761,1.0845290648838053,-0.18628912402400863,0.014661374796922139,-1.075569469436683,0.6422520742168762,-

In [46]:
code_text = decoded_output.split('```python')[-1].split("```\n\n")[0]

In [47]:
code_text

'\nimport statsmodels.api as sm\nimport numpy as np\n\n# Create a data structure\ndata = np.array([[3.60267822, 0.65837204],\n                [0.84795305, 0.35532686],\n                [-3.5086588, 0.78935562],\n                [0.9222189, -0.61641011],\n                [-3.28382469, -1.84424358],\n                [2.66905812, 0.12742175],\n                [0.52932274, -1.65113893],\n                [1.78450414, 0.13932657],\n                [1.74060050, 0.52932274],\n                [2.12194866, 2.5185468],\n                [3.08619872, -0.59866233],\n                [-0.59866233, -1.46342573],\n                [2.32934551, 2.24614075],\n                [0.54422228, 0.49171283],\n                [0.49171283, 0.02502064]])\n\n# Maximum likelihood estimation\nresults = sm.OLS(y=data[:, 0], x=data[:, 1]).fit()\n\n# Print the results\nprint(results.params)\n```'

In [51]:
code_output, CODE_STATUS = process_code(code_text, return_shell_output=False)


import statsmodels.api as sm
import numpy as np

# Create a data structure
data = np.array([[3.60267822, 0.65837204],
                [0.84795305, 0.35532686],
                [-3.5086588, 0.78935562],
                [0.9222189, -0.61641011],
                [-3.28382469, -1.84424358],
                [2.66905812, 0.12742175],
                [0.52932274, -1.65113893],
                [1.78450414, 0.13932657],
                [1.74060050, 0.52932274],
                [2.12194866, 2.5185468],
                [3.08619872, -0.59866233],
                [-0.59866233, -1.46342573],
                [2.32934551, 2.24614075],
                [0.54422228, 0.49171283],
                [0.49171283, 0.02502064]])

# Maximum likelihood estimation
results = sm.OLS(y=data[:, 0], x=data[:, 1]).fit()

# Print the results
print(results.params)
```
Command 'timeout 7 /usr/bin/python3 code.py' returned non-zero exit status 1. shell_output


TypeError: cannot unpack non-iterable int object

In [52]:
code_output, returned_code_status = process_code(code_text, return_shell_output=False)


import statsmodels.api as sm
import numpy as np

# Create a data structure
data = np.array([[3.60267822, 0.65837204],
                [0.84795305, 0.35532686],
                [-3.5086588, 0.78935562],
                [0.9222189, -0.61641011],
                [-3.28382469, -1.84424358],
                [2.66905812, 0.12742175],
                [0.52932274, -1.65113893],
                [1.78450414, 0.13932657],
                [1.74060050, 0.52932274],
                [2.12194866, 2.5185468],
                [3.08619872, -0.59866233],
                [-0.59866233, -1.46342573],
                [2.32934551, 2.24614075],
                [0.54422228, 0.49171283],
                [0.49171283, 0.02502064]])

# Maximum likelihood estimation
results = sm.OLS(y=data[:, 0], x=data[:, 1]).fit()

# Print the results
print(results.params)
```
Command 'timeout 7 /usr/bin/python3 code.py' returned non-zero exit status 1. shell_output


TypeError: cannot unpack non-iterable int object

In [50]:
code_output

-1